In [1]:
import polars as pl
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from xgboost import XGBRegressor, XGBClassifier
import skopt

import sys
sys.path.append("/Users/leon/Desktop/kaggle")
from src.rainfall.transformer_impl import RainfallTransformer as RT

In [2]:
train_df = pl.read_csv("./data/train.csv")
test_df = pl.read_csv("./data/test.csv")

In [6]:
rt = RT()
data = rt.transform(train_df)
X = data[:, :-1]
y = data[:, -1].ravel()
X_test = rt.transform(test_df)

shape: (5, 14)
┌─────┬──────────┬─────────┬─────────────┬───┬───────────┬──────────┬──────────┬──────────┐
│ id  ┆ pressure ┆ maxtemp ┆ temparature ┆ … ┆ windspeed ┆ day_sin  ┆ day_cos  ┆ rainfall │
│ --- ┆ ---      ┆ ---     ┆ ---         ┆   ┆ ---       ┆ ---      ┆ ---      ┆ ---      │
│ i64 ┆ f64      ┆ f64     ┆ f64         ┆   ┆ f64       ┆ f64      ┆ f64      ┆ i64      │
╞═════╪══════════╪═════════╪═════════════╪═══╪═══════════╪══════════╪══════════╪══════════╡
│ 0   ┆ 1017.4   ┆ 21.2    ┆ 20.6        ┆ … ┆ 17.2      ┆ 0.017213 ┆ 0.999852 ┆ 1        │
│ 1   ┆ 1019.5   ┆ 16.2    ┆ 16.9        ┆ … ┆ 21.9      ┆ 0.034422 ┆ 0.999407 ┆ 1        │
│ 2   ┆ 1024.1   ┆ 19.4    ┆ 16.1        ┆ … ┆ 18.1      ┆ 0.05162  ┆ 0.998667 ┆ 1        │
│ 3   ┆ 1013.4   ┆ 18.1    ┆ 17.8        ┆ … ┆ 35.6      ┆ 0.068802 ┆ 0.99763  ┆ 1        │
│ 4   ┆ 1021.8   ┆ 21.3    ┆ 18.4        ┆ … ┆ 24.8      ┆ 0.085965 ┆ 0.996298 ┆ 0        │
└─────┴──────────┴─────────┴─────────────┴───┴───────────┴───────

In [ ]:
sss = ms.StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
for train_index, eval_index in sss.split(X, y):
	X_train, X_eval = X[train_index], X[eval_index]
	y_train, y_eval = y[train_index], y[eval_index]

	xgb_clf = XGBClassifier(
		device="cuda",
		max_depth=6,
		colsample_bytree=0.9,
		subsample=0.9,
		n_estimators=10_000,
		learning_rate=0.1,
		eval_metric="auc",
		early_stopping_rounds=1000,
		alpha=1,
	).fit(X_train, y_train, eval_set=[(X_eval, y_eval)])


In [9]:
from sklearn.svm import SVC
from skopt import BayesSearchCV
import skopt.space as ss
svc_clf = SVC(kernel="poly")
search_spaces = {
	"C": ss.Real(0.1, 6.0),
	"kernel": ["linear", "poly", "rbf", "sigmoid"],
	"gamma": ["scale", "auto"],
}
opt = BayesSearchCV(svc_clf, search_spaces, cv=5, n_iter=100, scoring="roc_auc")
opt.fit(X, y)

BayesSearchCV(cv=5, estimator=SVC(kernel='poly'), n_iter=100, scoring='roc_auc',
              search_spaces={'C': Real(low=0.1, high=6.0, prior='uniform', transform='normalize'),
                             'gamma': ['scale', 'auto'],
                             'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [10]:
from sklearn.metrics import r2_score
y_pred = opt.predict(X)
r2_score(y, y_pred)

0.26016835016835016

In [7]:
res = xgb_clf.predict_proba(X_test)
with open(file="./submission.csv", mode="w") as f:
	f.write("id,rainfall\n")
	for i, r in enumerate(res):
		f.write(f"{i+2190},{r[1]:.6f}\n")

In [6]:
91291

91291